Assignment 7: Neural Networks using Keras and Tensorflow Please see the associated document for questions

If you have problems with Keras and Tensorflow on your local installation please make sure they are updated. On Google Colab this notebook runs.

In [ ]:
pip install tensorflow

In [1]:
# imports
from __future__ import print_function
import keras
from keras import utils as np_utils
import tensorflow
from keras.datasets import mnist
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam


In [2]:
# Hyper-parameters data-loading and formatting

batch_size = 128
num_classes = 10
epochs = 40

img_rows, img_cols = 28, 28

(x_train, lbl_train), (x_test, lbl_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

**Preprocessing**

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_test.shape)
x_train /= 255
x_test /= 255

y_train = keras.utils.np_utils.to_categorical(lbl_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(lbl_test, num_classes)


(10000, 28, 28, 1)


In [4]:
#CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (7, 7), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

#Even if adam learning rate for dedault was 0.001 we still defining it here
optimizer_a = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer_a,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

#Train
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

#Evaluate
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 22, 22, 32)        1600      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3872)              0         
                                                                 
 dense (Dense)               (None, 128)               495744    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 498,634
Trainable params: 498,634
Non-trainable params: 0
__________________________________________________

In [ ]:

## Define model ##
model = Sequential()

model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(300, activation = 'relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=tensorflow.keras.optimizers.SGD(learning_rate = 0.1),
        metrics=['accuracy'],)

fit_info = model.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy {}'.format(score[0], score[1]))

In [ ]:
plt.plot(fit_info.history['accuracy'], label=f'Training Accuracy (Epoch {epochs})')
plt.plot(fit_info.history['val_accuracy'], label=f'Validation Accuracy (Epoch {epochs})')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Define regularization factors to explore
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
# 0.00005, 0.00001, 0.0005, 0.0001
regularization_factors = [0.00051]
num_replicates = 3
val_accuracies = []

# Train networks with weight decay for different regularization factors and replicates
for i in regularization_factors:
    replicate_accuracies = []

    for replicate in range(num_replicates):
        # Build the neural network with weight decay
        model = Sequential()
        
        model.add(Flatten())
        model.add(Dense(500, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(i)))
        model.add(Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(i)))
        model.add(Dense(10, activation='softmax'))

        # Compile the model
        model.compile(
            loss='categorical_crossentropy',
            optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.1),
            metrics=['accuracy']
        )

        # Train the model
        history = model.fit(x_train, y_train, epochs=40, validation_data=(x_test, y_test), verbose=1)
       
        # Evaluate and store validation accuracy
        _, val_accuracy = model.evaluate(x_test, y_test, verbose=0)
        replicate_accuracies.append(val_accuracy)

    # Store mean and standard deviation of validation accuracy for the current regularization factor
    val_accuracies.append((np.mean(replicate_accuracies), np.std(replicate_accuracies)))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy {}'.format(score[0], score[1]))

In [ ]:
# Plot final validation accuracy with standard deviation as a function of the regularization factor
val_means, val_stds = zip(*val_accuracies)

plt.errorbar(regularization_factors, val_means, yerr=val_stds, marker='o', linestyle='-', label='Validation Accuracy')
plt.xscale('log')
  # Use a log scale for better visualization
plt.xlabel('Regularization Factor')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy with Weight Decay')
plt.legend()
plt.show()